In [1]:
import io
import random
import string
import warnings
import numpy as np
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog, messagebox
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer
from datetime import datetime
import pyttsx3
import speech_recognition as sr
from fpdf import FPDF
from googletrans import Translator
import pyaudio

warnings.filterwarnings('ignore')
nltk.download('popular', quiet=True)
lang_translator = Translator()

# Load and process corpus
with open('C:\\Users\\Asus\\Desktop\\SEM-8\\FDTL.DATA SCIENCE\\bots\\admission_bot_large.txt', 'r', encoding='utf8', errors='ignore') as file:
    corpus_text = file.read().lower()

tokenized_sentences = nltk.sent_tokenize(corpus_text)
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(word_list):
    return [lemmatizer.lemmatize(word) for word in word_list]

def normalize_text(input_text):
    remove_punct_map = str.maketrans('', '', string.punctuation)
    return lemmatize_tokens(nltk.word_tokenize(input_text.lower().translate(remove_punct_map)))

GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up", "hey",)
GREETING_REPLIES = ["Hi there!", "Hello!", "Greetings!", "Hey! Ask me about admissions"]

def check_greeting(text):
    for term in text.split():
        if term.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_REPLIES)
    return None

def generate_bot_reply(user_input_text):
    user_input_text = user_input_text.lower().strip()
    tokenized_sentences.append(user_input_text)
    vectorizer = TfidfVectorizer(tokenizer=normalize_text, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(tokenized_sentences)
    similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix)
    relevant_indices = similarity_scores.argsort()[0][-3:-1]

    sorted_scores = similarity_scores.flatten()
    sorted_scores.sort()
    matched_responses = [tokenized_sentences[i] for i in relevant_indices if sorted_scores[-2] > 0]

    tokenized_sentences.remove(user_input_text)
    return "\n- ".join(matched_responses) if matched_responses else "I'm sorry! I don't have info on that. Try another question about admissions."

def handle_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as mic_source:
        try:
            feedback_label.config(text="Listening...")
            voice_data = recognizer.listen(mic_source)
            recognized_text = recognizer.recognize_google(voice_data)
            feedback_label.config(text="Processing...")
            process_user_query(recognized_text)
        except sr.UnknownValueError:
            messagebox.showerror("Voice Error", "Could not understand your voice input.")
        except sr.RequestError:
            messagebox.showerror("Network Error", "Network error. Please check your connection.")
        feedback_label.config(text="")

def read_aloud(response_text):
    tts_engine = pyttsx3.init()
    tts_engine.setProperty('rate', 150)
    tts_engine.say(response_text)
    tts_engine.runAndWait()

def export_chat_to_txt():
    save_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
    if save_path:
        chat_log = chat_display.get("1.0", tk.END).split("\n")
        with open(save_path, "w", encoding="utf-8") as file:
            for line in chat_log:
                if line.strip():
                    file.write(line + "\n")

def process_user_query(event=None):
    message = user_entry.get().strip()
    if not message:
        return

    current_time = datetime.now().strftime("[%H:%M:%S]")
    chat_display.insert(tk.END, f"\n👤 You ({current_time}): {message}", "user")
    user_entry.delete(0, tk.END)

    chatbot_reply = check_greeting(message) or generate_bot_reply(message)
    chat_display.insert(tk.END, f"\n🤖 Chatbot ({current_time}): {chatbot_reply}", "bot")
    chat_display.yview(tk.END)
    app_window.after(1000, lambda: read_aloud(chatbot_reply))

def reset_chat_display():
    chat_display.delete('1.0', tk.END)

# GUI setup
app_window = tk.Tk()
app_window.title("University Admission Assistant")
app_window.geometry("1920x1080")
app_window.configure(bg="#2c3e50")

title_label = tk.Label(app_window, text="🎓 Admission Assistant", font=("unispace", 18, "bold"), bg="#34495e", fg="#ecf0f1", pady=10)
title_label.pack(fill=tk.X)

chat_display = scrolledtext.ScrolledText(app_window, wrap=tk.WORD, width=50, height=20, font=("uinspace", 12), bg="#ecf0f1", fg="#2c3e50")
chat_display.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
chat_display.tag_config("user", foreground="blue")
chat_display.tag_config("bot", foreground="red")

feedback_label = tk.Label(app_window, text="", font=("unispace", 10), bg="#2c3e50", fg="white")
feedback_label.pack()

input_section = tk.Frame(app_window, pady=5, bg="#2c3e50")
input_section.pack(fill=tk.X)

user_entry = tk.Entry(input_section, width=30, font=("unispace", 12), bg="#ffffff", fg="#000000", bd=2, relief=tk.FLAT)
user_entry.pack(side=tk.LEFT, padx=10, pady=5, expand=True, fill=tk.X)
user_entry.bind("<Return>", process_user_query)

send_btn = ttk.Button(input_section, text="Send", command=process_user_query)
send_btn.pack(side=tk.RIGHT, padx=5)

mic_btn = ttk.Button(input_section, text="🎤", command=handle_voice_input)
mic_btn.pack(side=tk.RIGHT, padx=5)

save_btn = ttk.Button(app_window, text="Save as TXT", command=export_chat_to_txt)
save_btn.pack()

clear_btn = ttk.Button(app_window, text="Clear Chat", command=reset_chat_display)
clear_btn.pack(pady=5)

app_window.mainloop()


In [16]:
import io
import random
import string
import warnings
import numpy as np
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog, messagebox
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer
from datetime import datetime
import pyttsx3
import threading
import speech_recognition as sr
from fpdf import FPDF
from googletrans import Translator
import pyaudio
import threading
import re
import time

# === INITIAL SETUP ===
warnings.filterwarnings('ignore')
nltk.download('popular', quiet=True)
lang_translator = Translator()

with open('C:\\Users\\Asus\\Desktop\\SEM-8\\FDTL.DATA SCIENCE\\bots\\admission_bot_large.txt', 'r', encoding='utf8', errors='ignore') as file:
    corpus_text = file.read().lower()

tokenized_sentences = nltk.sent_tokenize(corpus_text)
lemmatizer = WordNetLemmatizer()

# === TEXT PROCESSING ===
def lemmatize_tokens(word_list):
    return [lemmatizer.lemmatize(word) for word in word_list]

def normalize_text(input_text):
    remove_punct_map = str.maketrans('', '', string.punctuation)
    return lemmatize_tokens(nltk.word_tokenize(input_text.lower().translate(remove_punct_map)))

GREETING_KEYWORDS = ("hello", "hi", "greetings", "sup", "what's up", "hey",)
GREETING_REPLIES = ["Hi there!", "Hello!", "Ram Ram How are you!", "Hey! Ask me about admissions"]

def check_greeting(text):
    for term in text.split():
        if term.lower() in GREETING_KEYWORDS:
            return random.choice(GREETING_REPLIES)
    return None

def generate_bot_reply(user_input_text):
    user_input_text = user_input_text.lower().strip()
    tokenized_sentences.append(user_input_text)
    vectorizer = TfidfVectorizer(tokenizer=normalize_text, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(tokenized_sentences)
    similarity_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix)
    relevant_indices = similarity_scores.argsort()[0][-3:-1]

    sorted_scores = similarity_scores.flatten()
    sorted_scores.sort()
    matched_responses = [tokenized_sentences[i] for i in relevant_indices if sorted_scores[-2] > 0]

    tokenized_sentences.remove(user_input_text)
    return "\n- ".join(matched_responses) if matched_responses else "I'm sorry! I don't have info on that. Try another question about admissions."

# === VOICE RECOGNITION ===
def handle_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as mic_source:
        try:
            feedback_label.config(text="Listening...")
            voice_data = recognizer.listen(mic_source)
            recognized_text = recognizer.recognize_google(voice_data)
            feedback_label.config(text="Processing...")
            process_user_query(recognized_text)
        except sr.UnknownValueError:
            messagebox.showerror("Voice Error", "Could not understand your voice input.")
        except sr.RequestError:
            messagebox.showerror("Network Error", "Network error. Please check your connection.")
        feedback_label.config(text="")

# === TTS CONTROL ===
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)
is_paused = False
stop_signal = False

def read_aloud(response_text):
    def speak():
        global stop_signal, is_paused
        words = response_text.split()
        for word in words:
            if stop_signal:
                stop_signal = False
                tts_engine.stop()
                return
            while is_paused:
                tts_engine.stop()
                threading.Event().wait(0.1)  # pause polling
            tts_engine.say(word)
            tts_engine.runAndWait()

    threading.Thread(target=speak, daemon=True).start()

def toggle_pause():
    global is_paused
    is_paused = not is_paused
    pause_btn.config(text="Resume" if is_paused else "Pause")

def skip_speech():
    global stop_signal
    stop_signal = True
    tts_engine.stop()

# === FILE EXPORT ===
def export_chat_to_txt():
    save_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
    if save_path:
        chat_log = chat_display.get("1.0", tk.END).split("\n")
        with open(save_path, "w", encoding="utf-8") as file:
            for line in chat_log:
                if line.strip():
                    file.write(line + "\n")

# === PROCESSING QUERIES ===
def process_user_query(event=None):
    if isinstance(event, str):
        message = event.strip()
    else:
        message = user_entry.get().strip()
        user_entry.delete(0, tk.END)

    if not message:
        return

    current_time = datetime.now().strftime("[%H:%M:%S]")
    chat_display.insert(tk.END, f"\n👤 You ({current_time}): {message}", "user")

    chatbot_reply = check_greeting(message) or generate_bot_reply(message)
    chat_display.insert(tk.END, f"\n🤖 Chatbot ({current_time}): {chatbot_reply}", "bot")
    chat_display.yview(tk.END)

    app_window.after(1000, lambda: read_aloud(chatbot_reply))

def reset_chat_display():
    chat_display.delete('1.0', tk.END)

# === GUI SETUP ===
app_window = tk.Tk()
app_window.title("University Admission Assistant")
app_window.geometry("600x700")
app_window.configure(bg="#2c3e50")

title_label = tk.Label(app_window, text="🎓 Admission Assistant", font=("unispace", 18, "bold"),
                       bg="#34495e", fg="#ecf0f1", pady=10)
title_label.pack(fill=tk.X)

chat_display = scrolledtext.ScrolledText(app_window, wrap=tk.WORD, width=50, height=20,
                                         font=("unispace", 12), bg="#ecf0f1", fg="#2c3e50")
chat_display.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
chat_display.tag_config("user", foreground="blue")
chat_display.tag_config("bot", foreground="red")

feedback_label = tk.Label(app_window, text="", font=("unispace", 10), bg="#2c3e50", fg="white")
feedback_label.pack()

input_section = tk.Frame(app_window, pady=5, bg="#2c3e50")
input_section.pack(fill=tk.X)

user_entry = tk.Entry(input_section, width=30, font=("unispace", 12), bg="#ffffff", fg="#000000", bd=2, relief=tk.FLAT)
user_entry.pack(side=tk.LEFT, padx=10, pady=5, expand=True, fill=tk.X)
user_entry.bind("<Return>", process_user_query)

send_btn = ttk.Button(input_section, text="Send", command=process_user_query)
send_btn.pack(side=tk.RIGHT, padx=5)

mic_btn = ttk.Button(input_section, text="🎤", command=handle_voice_input)
mic_btn.pack(side=tk.RIGHT, padx=5)

save_btn = ttk.Button(app_window, text="Save as TXT", command=export_chat_to_txt)
save_btn.pack()

clear_btn = ttk.Button(app_window, text="Clear Chat", command=reset_chat_display)
clear_btn.pack(pady=5)

# === CONTROL BUTTONS FOR TTS ===
controls_frame = tk.Frame(app_window, bg="#2c3e50")
controls_frame.pack()

pause_btn = ttk.Button(controls_frame, text="Pause", command=toggle_pause)
pause_btn.pack(side=tk.LEFT, padx=5)

skip_btn = ttk.Button(controls_frame, text="Skip", command=skip_speech)
skip_btn.pack(side=tk.LEFT, padx=5)

# === START GUI LOOP ===
app_window.mainloop()


Exception in thread Thread-20 (speak):
Traceback (most recent call last):
  File "C:\Users\Asus\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\Asus\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Asus\AppData\Local\Temp\ipykernel_107408\3203878739.py", line 128, in speak
  File "C:\Users\Asus\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-21 (speak):
Traceback (most recent call last):
  File "C:\Users\Asus\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\Asus\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Asus\AppData\Local\Temp\ipykernel_107408\3203878739.py", line 128, in speak
  File "C:\Users\Asus\anaconda3\Lib\site-packages\pyttsx3\